# Convolution and Pool Layers-v2

In [ ]:
#! pip install Pillow

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
#import cv2

In [ ]:
from PIL import Image
jpgfile = Image.open('imgs/shelf.JPG')

print(jpgfile.bits, jpgfile.size, jpgfile.format)
plt.imshow(jpgfile, cmap='gray')
plt.show()

gray_img = np.array(jpgfile).sum(2).astype('float64')/(3*255)
plt.imshow(gray_img , cmap='gray')
plt.show()

In [ ]:
gray_img.shape

In [ ]:
bgr_img = cv2.imread('imgs/shelf.JPG')
bgr_img = cv2.resize(bgr_img, (0,0), fx=0.25, fy=0.25) 
gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY).astype("float32")/255

plt.imshow(gray_img, cmap='gray')
plt.show()

## Convolution Layer

Suppose that out image looks like that
![Image](imgs/image_cl.png)
and filter
![Image](imgs/filter_cl.png)
Then applying filter means
![Applaying Filters](imgs/applying_cl.gif)

### Filters

In [ ]:
import numpy as np

filter_1 = np.array([[-1, -1, 1, 1], [-1, -1, 1, 1], [-1, -1, 1, 1], [-1, -1, 1, 1]])
filter_2 = np.array([[-1, -1, -1, -1], [-1, -1, -1, -1], [1, 1, 1, 1], [1, 1, 1, 1]])
filter_3 = np.array([[0, -1, 0, 0], [-1, 21, -1, 0], [0, -1, 0, 0], [0, 0, 0, 0]])
filter_4 = np.ones((4, 4))
filters = np.array([filter_1, filter_2, filter_3, filter_4])

filters

### Change shape for expexted by pytorch

In [ ]:
filters_torch = torch.from_numpy(filters).unsqueeze(1).type(torch.FloatTensor)
filters_params = torch.nn.Parameter(filters_torch)
filters_params.shape

Shape of filters in pytorch

`[ number of output chanels, number of input chanels, hight, width]` 

### Change shape of image as well

In [ ]:
gray_img_torch = torch.from_numpy(gray_img).unsqueeze(0).unsqueeze(1).type(torch.FloatTensor)
gray_img.shape, gray_img_torch.shape

In [ ]:
gray_img_torch

### Model: we will expain it later

Here it is just to show how filters works!

In [ ]:
class Net(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size):
        super(Net, self).__init__()        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, bias=False)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        conv_x = self.conv(x)
        activated_x = F.relu(conv_x)
        pool_x = self.pool(activated_x)
        
        return conv_x, activated_x, pool_x

## Initialize model
model = Net(3, 4, (4, 4))
## Initialize weights
model.conv.weight = filters_params

# print out the layer in the network
print(model)

### Let's look at output of layers

##### Convolution layer

In [ ]:
conv_layer, activated_layer, pooled_layer = model(gray_img_torch)
conv_layer.shape, activated_layer.shape

In [ ]:
def viz_layer(layer):
    n_filters = layer.shape[1]
    fig = plt.figure(figsize=(20, 20))
    
    for i in range(n_filters):
        ax = fig.add_subplot(1, n_filters, i+1, xticks=[], yticks=[])
        # grab layer outputs
        ax.imshow(np.squeeze(layer[0,i].data.numpy()), cmap='gray')
        ax.set_title('Output %s' % str(i+1))

#### Output before activation

In [ ]:
viz_layer(conv_layer)

#### Output after activation

In [ ]:
viz_layer(activated_layer)

### MaxPool Layer

![Pooling](imgs/pool.jpeg)

![Pooling](imgs/maxpool.jpeg)

In [ ]:
viz_layer(pooled_layer)

In [ ]:
print("Before: ", activated_layer.shape)
print("After: ", pooled_layer.shape)

Links
* <https://adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/>
* <http://cs231n.github.io/convolutional-networks/>
* <http://udacity.com>